# Ejemplo prompting para usar arquitectura zero-shot empleando LLMs



In [0]:
from loguru import logger
from mlflow.deployments import get_deploy_client

from requests.exceptions import HTTPError

In [0]:
prompt = [
    "SYSTEM: Eres un asistente que analiza un texto y devuelve la temática de dicho texto\n\n",
    "USER: A partir de esta cuestión {business_question}, devuelve una de las siguientes categorías: 'Otro', 'Deporte', 'Finanzas', 'Informática', 'Cine' \n\n",
    "INSTRUCTIONS: \n\n",
    "Presta atención a la cuestión y a las temáticas planteadas. Devuelve solo una única temática\n\n",
    "Ejemplo 1 de uso\n\n",
    "Pregunta: Las acciones del Santander han bajado un 2% en los últimos tres meses",
    "Respuesta: Finanzas\n\n",
    "Ejemplo 2 de uso\n\n",
    "Pregunta: ¿España ganará la Eurocopa este año?",
    "Respuesta: Deporte\n\n",
    "Pregunta: Soy un enamorado de python",
    "Respuesta: Informática\n\n",
    "Pregunta: El Señor de los Anillos es una película que está sobrevalorada",
    "Respuesta: Cine\n\n"
]

In [0]:
def _call_llm_model(
    prompt: str, model_name: str = "paa-gpt-4o-v20240513-dedicado-analitica"
    ) -> str:
    """
    Función que llama al modelo LLM para hacer una predicción a partir de un prompt

    Args:
        prompt (str): Mensaje del prompt

    Returns:
        response (str): Respuesta del modelo si no hay fallo en la conexión.
        En otro caso, devuelve el mensaje de error
    """
    try:
        client = get_deploy_client("databricks")
        response = client.predict(
            endpoint=model_name,
            inputs={"messages": [{"role": "user", "content": prompt}]}
        )
        response = response["choices"][0]["message"]["content"]
        logger.info("Ejecución del LLM realizada")
    except HTTPError as e:
        if e.response.text:
            logger.info(f"{e}. Response text: {e.response.text}")
        else:
            logger.info(e)
        response = e
    return response

In [0]:
def _add_business_question_prompt(prompt: str, business_question: str) -> str:
  """
  Función que añade al prompt la cuestión de negocio

    Args:
        prompt (str): Mensaje del prompt
        business_question (str): Pregunta de negocio a clasificar

    Returns:
        response (str): Respuesta del modelo si no hay fallo en la conexión.
        En otro caso, devuelve el mensaje de error
  """
  prompt_end = " ".join(prompt)
  prompt_end = prompt_end.replace("{business_question}", business_question)
  return prompt_end

## Ejemplo de uso

In [0]:
business_question = "Soy fanático de El Quijote"
prompt_end = _add_business_question_prompt(prompt, business_question)
_call_llm_model(prompt_end)

2024-06-28 20:24:01.574 | INFO     | __main__:_call_llm_model:21 - Ejecución del LLM realizada


'Otro'

In [0]:
business_question = "El Madrid ganó la última Champions"
prompt_end = _add_business_question_prompt(prompt, business_question)
_call_llm_model(prompt_end)

2024-06-28 20:24:02.673 | INFO     | __main__:_call_llm_model:21 - Ejecución del LLM realizada


'Deporte'

In [0]:
business_question = "Me encanta ver la televisión por la noche"
prompt_end = _add_business_question_prompt(prompt, business_question)
_call_llm_model(prompt_end)

2024-06-28 20:24:03.035 | INFO     | __main__:_call_llm_model:21 - Ejecución del LLM realizada


'Cine'